======================================================================================================

Disclaimer: Parts of this notebook are adapted from [this](https://github.com/DeepRLCourse/Workshop-6-Material/blob/main/Workshop6_Notebook.ipynb) GitHub repo with minor modifications. Credit to the original authors.

======================================================================================================

# Workshop 2 : Multi-Armed Bandits

In this notebook, we will explore the core concepts of **multi-armed bandits**, focusing on:

1. Definitions and properties.
2. Action-value methods and uncertainty in bandits.
3. Exploration-exploitation strategy:
   - $\epsilon$-greedy

4. Real-world applications:
   - Recommender Systems
   - Search

We'll show **pure Python implementations** and **Gymnasium-based** implementations, so you can see how each algorithm might look in a more standardized RL environment setup.


> **Note:** In typical RL frameworks, a bandit is treated as either a single-step environment (reset after each pull) or a multi-step environment with a fixed horizon. The examples below choose a **multi-step environment with a fixed horizon** to mimic `num_steps` pulls in a single episode.



By the end of this notebook, you should:
- Understand the **multi-armed bandit problem** and how it differs from full RL (non-associative property).
- Implement the exploration method of ($\epsilon$-greedy), both purely and via Gymnasium.
- See how these ideas apply to **recommender systems** and **search**.

## 1. Setup and Imports

We'll import:
- `numpy` for numerical operations
- `matplotlib` for plotting
- `gymnasium` to demonstrate how to wrap the bandit environment in the standard Gym interface

We also set a random seed for reproducibility.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from gymnasium import spaces

np.random.seed(42)

## 2. Multi-Armed Bandits: Recap

### 2.1 Definition of the Problem and Non-Associativity Property


A **multi-armed bandit (MAB)** is a fundamental reinforcement learning setting modeling **decision-making under uncertainty**. The problem is often described as having $k$ different slot machines (arms), each with an **unknown** distribution of rewards.

- We have **$ k $** arms/actions.
- Each arm $ a $ has an **unknown expected reward** $ q_*(a) $:
  $$
  q_*(a) = \mathbb{E}[R \mid A = a],
  $$
  where $ R $ is the reward for choosing action $ A $.
- We want to **maximize the cumulative reward**:
  $$
  G = \sum_{t=1}^{T} R_t.
  $$
- We **do not** know $ q_*(a) $ and must **estimate** these values over time.

**Non-Associativity**:
- Unlike a full RL problem, multi-armed bandits are **non-associative**—there are no distinct states or transitions.
- The best action does not depend on any "state"; the challenge is figuring out which action yields the highest expected reward overall.

### 2.2 Action-Value Methods and Types


We define the **action-value** $ Q_t(a) $ as our estimate of $ q_*(a) $. Using **sample-average methods**:
$$
Q_t(a) = \frac{1}{N_t(a)} \sum_{i=1}^{N_t(a)} R_i,
$$
where $ N_t(a) $ is the number of times action $ a $ has been chosen up to time $ t $, and $ R_i $ are the observed rewards.

**Incremental Update Rule** (to avoid storing entire histories):
$$
Q_{t+1}(a) = Q_t(a) + \frac{1}{N_t(a)}(R_t - Q_t(a)).
$$

#### Constant Step-Size for Nonstationary Problems
When rewards change over time, we use **constant step-size** $ \alpha $:
$$
Q_{t+1}(a) = Q_t(a) + \alpha (R_t - Q_t(a)).
$$
- Large $\alpha$ gives more weight to recent data, useful for **nonstationary** environments.

### 2.3 Exploration-Exploitation Dilemma and Uncertainty


A core challenge is balancing **exploration** and **exploitation**:
- **Exploration**: try different actions to reduce uncertainty in their value estimates.
- **Exploitation**: select the best-known action to maximize immediate reward.

Mathematically, if an action is not selected often, $ N_t(a) $ remains small, so the **uncertainty** in its estimate is large. An agent must ensure all actions are sufficiently explored to accurately approximate $ q_*(a) $.

## 3. Exploration in Bandits

We now discuss the exploration strategies in MAB. We'll first show **plain Python** implementations, then **Gymnasium-based** versions.

### 3.1 $\epsilon$-Greedy

From the recitation:

> **$\epsilon$-greedy**:
> $$
> A_t =
> \begin{cases}
> \arg\max_a Q_t(a), & \text{with probability } 1-\epsilon \\
> \text{random } a, & \text{with probability } \epsilon.
> \end{cases}
> $$

- Simple to implement.
- Guarantees each arm is sampled (with probability $\epsilon$).
- Can be inefficient if $\epsilon$ is large because it spends too much time on suboptimal arms.

#### 3.1.1 Plain Python: Bernoulli Bandit and $\epsilon$-Greedy

In [ ]:
class BernoulliBandit:
    """
    A simple k-armed Bernoulli bandit.
    Each arm i has a probability p[i] of returning reward=1.
    """
    def __init__(self, p):
        self.p = np.array(p)
        self.k = len(p)

    def step(self, action):
        # Returns 1 with probability p[action], else 0
        return 1 if np.random.rand() < self.p[action] else 0

def epsilon_greedy(bandit, num_steps=1000, epsilon=0.1):
    """
    Epsilon-greedy action selection.
    bandit: an instance of BernoulliBandit
    num_steps: total number of pulls
    epsilon: exploration rate
    """
    k = bandit.k
    Q = np.zeros(k) # Estimated values of arms
    N = np.zeros(k) # Count of pulls for each arm

    rewards_history = []

    for _ in range(num_steps):
        # Explore vs Exploit
        if np.random.rand() < epsilon:
            action = np.random.randint(k) # Explore
        else:
            action = np.argmax(Q) # Exploit

        reward = bandit.step(action)
        N[action] += 1
        Q[action] += (reward - Q[action]) / N[action] # Update estimate
        rewards_history.append(reward)

    return np.array(rewards_history), Q

# Demo (plain Python)
p = [0.1, 0.3, 0.8]
bandit = BernoulliBandit(p)
rewards, Q_est = epsilon_greedy(bandit, num_steps=1000, epsilon=0.1)
print(f"Estimated Q-values: {Q_est}")
print(f"Total reward: {rewards.sum()} out of 1000 pulls")

Estimated Q-values: [0.0625     0.36585366 0.80241493]
Total reward: 749 out of 1000 pulls


#### 3.1.2 Gymnasium-Based Implementation: $\epsilon$-Greedy

Below, we wrap the Bernoulli bandit logic in a Gymnasium environment. In this approach:
- We have a single episode of length `num_steps`.
- The environment’s `action_space` is `Discrete(k)`.
- Observations are trivial (we’ll just return a dummy array).
- After each `step(action)`, we increment an internal step count and eventually set `done=True` when `num_steps` is reached.

Then we run an $\epsilon$-greedy policy in the **standard Gym loop**.

In [ ]:
class BernoulliBanditEnv(gym.Env):
    """
    Gymnasium environment for a k-armed Bernoulli bandit.
    The episode length is fixed to num_steps.
    """
    def __init__(self, p, num_steps=1000):
        super().__init__()
        self.p = np.array(p)
        self.k = len(p)
        self.num_steps = num_steps

        # Define action & observation space
        self.action_space = spaces.Discrete(self.k)
        # We'll return a dummy observation (e.g., shape (1,))
        self.observation_space = spaces.Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32)

        self.current_step = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        # Return a dummy observation
        return np.array([0.0], dtype=np.float32), {}

    def step(self, action):
        reward = 1 if np.random.rand() < self.p[action] else 0
        self.current_step += 1
        done = (self.current_step >= self.num_steps)
        # We don't have a meaningful observation, so just return a dummy
        obs = np.array([0.0], dtype=np.float32)
        info = {}
        return obs, float(reward), done, False, info

# Gym-based epsilon-greedy
def epsilon_greedy_gym(env, epsilon=0.1):
    k = env.action_space.n
    Q = np.zeros(k)
    N = np.zeros(k)
    rewards_history = []

    obs, info = env.reset()
    done = False
    step_count = 0

    while not done:
        step_count += 1
        # Explore vs Exploit
        if np.random.rand() < epsilon:
            action = np.random.randint(k)
        else:
            action = np.argmax(Q)

        obs, reward, done, truncated, info = env.step(action)

        N[action] += 1
        Q[action] += (reward - Q[action]) / N[action]
        rewards_history.append(reward)

        if done or truncated:
            break

    return np.array(rewards_history), Q

# Demo (Gym)
p = [0.1, 0.3, 0.8]
env = BernoulliBanditEnv(p, num_steps=1000)
gym_rewards, gym_Q = epsilon_greedy_gym(env, epsilon=0.1)
print(f"[GYM] Estimated Q-values: {gym_Q}")
print(f"[GYM] Total reward: {gym_rewards.sum()} out of 1000 pulls")

[GYM] Estimated Q-values: [0.0625     0.33962264 0.80294451]
[GYM] Total reward: 731.0 out of 1000 pulls


## 4. Bandit Applications
In this section, we provide **practical examples** for two major real-world applications of bandits:
- Recommender Systems
- Search

We’ll show both **pure Python** and **Gymnasium** approaches.

### 4.1 Recommender Systems

A recommender system often picks an item (action) to recommend for a particular user, receives feedback (reward), and aims to maximize user satisfaction (clicks, watch time, etc.).

#### 4.1.1 Pure Python Example [EXTRA]
We'll simulate a scenario where we have `k` items to recommend, and each user is described by a **context** vector. We'll keep it simple:
- Each item has a weight vector.
- The user context is sampled randomly.
- The probability of a click (reward=1) is `sigmoid(w_item . context)`.
We'll implement a minimal function to **run an $\epsilon$-greedy** approach for demonstration.

In [ ]:
class RecommenderSystemBandit:
    """
    Pure Python bandit environment for a recommender system.
    """
    def __init__(self, item_weights, context_dim=2, num_users=1000):
        """
        item_weights: np.array of shape (k, context_dim)
        context_dim: dimension of each user's feature vector
        num_users: how many user interactions we simulate
        """
        self.item_weights = item_weights
        self.k = item_weights.shape[0]
        self.context_dim = context_dim
        self.num_users = num_users
        self.current_step = 0

    def get_context(self):
        # Sample a random user feature vector
        return np.random.randn(self.context_dim)

    def step(self, action, user_context):
        """
        Probability of reward = sigmoid(item_weights[action] . user_context)
        """
        w = self.item_weights[action]
        p = 1.0 / (1.0 + np.exp(- w.dot(user_context)))
        reward = 1 if np.random.rand() < p else 0
        self.current_step += 1
        done = (self.current_step >= self.num_users)
        return reward, done

def run_epsilon_greedy_recommender(env, epsilon=0.1):
    k = env.k
    Q = np.zeros(k)
    # The number of times each 'action' has been selected,
    # each element in N corresponds to a particular arm.
    N = np.zeros(k)
    rewards_history = []

    while True:
        context = env.get_context()
        if np.random.rand() < epsilon:
            action = np.random.randint(k)
        else:
            action = np.argmax(Q)

        reward, done = env.step(action, context)
        N[action] += 1
        Q[action] += (reward - Q[action]) / N[action]
        rewards_history.append(reward)

        if done:
            break

    return np.array(rewards_history), Q

# Demo: RecommenderSystemBandit with k=3 items
item_weights = np.array([
    [1.5, 0.5],   # item 0
    [-0.5, 1.2], # item 1
    [0.8, 0.8]   # item 2
])
rec_env = RecommenderSystemBandit(item_weights, context_dim=2, num_users=1000)
recomm_rewards, recomm_Q = run_epsilon_greedy_recommender(rec_env, epsilon=0.1)
print("Recommender (pure Python) total reward:", recomm_rewards.sum())
print("Final Q estimates:", recomm_Q)

Recommender (pure Python) total reward: 476
Final Q estimates: [0.46927374 0.48042705 0.47592593]


#### 4.1.2 Gymnasium Example [EXTRA]
We create a Gym environment where each step corresponds to recommending an item for a randomly sampled user context. We’ll do a short **$\epsilon$-greedy** run.

In [ ]:
class RecommenderSystemGymEnv(gym.Env):
    def __init__(self, item_weights, context_dim=2, num_users=1000):
        super().__init__()
        self.item_weights = item_weights
        self.k = item_weights.shape[0]
        self.context_dim = context_dim
        self.num_users = num_users
        self.current_step = 0

        self.action_space = spaces.Discrete(self.k)
        # Observation is the user context
        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(self.context_dim,), dtype=np.float32)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.context = np.random.randn(self.context_dim).astype(np.float32)
        return self.context, {}

    def step(self, action):
        w = self.item_weights[action]
        p = 1.0 / (1.0 + np.exp(- w.dot(self.context)))
        reward = 1.0 if np.random.rand() < p else 0.0
        self.current_step += 1
        done = (self.current_step >= self.num_users)
        info = {}

        # sample next user context
        self.context = np.random.randn(self.context_dim).astype(np.float32)
        return self.context, reward, done, False, info

def run_epsilon_greedy_recommender_gym(env, epsilon=0.1):
    k = env.action_space.n
    Q = np.zeros(k)
    N = np.zeros(k)
    rewards_history = []

    obs, info = env.reset()
    done = False
    while not done:
        if np.random.rand() < epsilon:
            action = np.random.randint(k)
        else:
            action = np.argmax(Q)

        new_obs, reward, done, truncated, info = env.step(action)
        N[action] += 1
        Q[action] += (reward - Q[action]) / N[action]
        rewards_history.append(reward)

        obs = new_obs
        if done or truncated:
            break

    return np.array(rewards_history), Q

# Demo: RecommenderSystemGymEnv
item_weights = np.array([
    [1.5, 0.5],   # item 0
    [-0.5, 1.2], # item 1
    [0.8, 0.8]   # item 2
])
env_rec = RecommenderSystemGymEnv(item_weights, context_dim=2, num_users=1000)
rec_gym_rewards, rec_gym_Q = run_epsilon_greedy_recommender_gym(env_rec, epsilon=0.1)
print("[GYM Recommender] total reward:", rec_gym_rewards.sum())
print("[GYM Recommender] final Q estimates:", rec_gym_Q)

[GYM Recommender] total reward: 517.0
[GYM Recommender] final Q estimates: [0.51923077 0.52070263 0.44680851]


### 4.2 Search

A simplified "search" bandit scenario might involve multiple **ranking strategies** or **candidate documents**. We pick which strategy or document to display for a given query, and get a click/no-click reward. Below, we provide a toy model:
- We have `k` different "search strategies."
- Each query is a context vector.
- Probability of a click is a logistic function of the chosen strategy’s weight.

#### 4.2.1 Pure Python Example [EXTRA]
We'll do the same structure as the Recommender, but we’ll call it "SearchBandit."

In [ ]:
class SearchBandit:
    """
    Pure Python environment to simulate 'search strategies' bandit.
    """
    def __init__(self, strategy_weights, context_dim=2, num_queries=500):
        self.strategy_weights = strategy_weights
        self.k = strategy_weights.shape[0]
        self.context_dim = context_dim
        self.num_queries = num_queries
        self.current_step = 0

    def get_context(self):
        return np.random.randn(self.context_dim)

    def step(self, action, query_context):
        w = self.strategy_weights[action]
        p = 1.0 / (1.0 + np.exp(- w.dot(query_context)))
        reward = 1 if np.random.rand() < p else 0
        self.current_step += 1
        done = (self.current_step >= self.num_queries)
        return reward, done

def run_thompson_search(env):
    """
    We'll demonstrate Thompson Sampling here.
    """
    k = env.k
    alpha = np.ones(k)
    beta = np.ones(k)
    rewards_history = []

    while True:
        query_context = env.get_context()
        sampled_q = np.random.beta(alpha, beta)
        action = np.argmax(sampled_q)
        reward, done = env.step(action, query_context)
        if reward == 1:
            alpha[action] += 1
        else:
            beta[action] += 1
        rewards_history.append(reward)

        if done:
            break

    return np.array(rewards_history), alpha, beta

# Demo: 3 search strategies
strategy_weights = np.array([
    [1.0, 1.0],   # strategy 0
    [-0.5, 2.0], # strategy 1
    [2.0, -0.2]  # strategy 2
])
search_env = SearchBandit(strategy_weights, context_dim=2, num_queries=500)
search_rewards, alpha_s, beta_s = run_thompson_search(search_env)
print("Search (pure Python) total reward:", search_rewards.sum())
print("Alpha:", alpha_s, "Beta:", beta_s)

Search (pure Python) total reward: 264
Alpha: [114.  68.  85.] Beta: [98. 69. 72.]
